Finetuning the model on financial_phrasebank dataset, that consists of pairs of text-labels to classify financial-related sentences, if they are either <span style="color: red;">positive</span>, <span style="color: purple;">neutral</span> or <span style="color: green;">negative</span>.

# 1.Experimental Setup1

## 1.1 Setup

## 1.2 Training

### 1.2.1 Run code on CPU version

In [ ]:
!pwd

In [ ]:
!python ../../../peft_train.py \
--model_name ../../../pretrain_models/flan-t5-xl \
--max_seq_len 128 \
--group_by_length \
--max_steps 200 \
--dataset_name ../../../text-classification/imdb \
--num_labels 2 \
--epochs 5 \
--learning_rate 1e-3 \
--per_device_train_batch_size 16 \
--per_device_eval_batch_size 16 \
--model_type SEQ_2_SEQ_LM \
--output_model_path ./result/flan-t5-xl-imdb-lora \
--bnb_4bit_compute_dtype float16 \
--use_4b False \
--use_cpu


Using the latest cached version of the module from /Users/wengbenjue/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--accuracy/f887c0aab52c2d38e1f8a215681126379eca617f96c447638f751434e8e65b14 (last modified on Mon Apr 24 01:17:15 2023) since it couldn't be found locally at evaluate-metric--accuracy, or remotely on the Hugging Face Hub.
use AutoModelForSeq2SeqLM load bert model.
trainable params: 2752512 || all params: 79713664 || trainable%: 3.4529989739274813
Running tokenizer on dataset: 100%|█| 4075/4075 [00:00<00:00, 24174.06 examples/
Running tokenizer on dataset: 100%|█| 453/453 [00:00<00:00, 22778.73 examples/s]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The c

### 1.2.2 Run code on GPU version
Load the model together with the adapter with few lines of code! Check the snippet below to load the adapter from the Hub and run the example evaluation.

In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb evaluate
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/llms-peft-cook-colab/experiments/flan-t5-xl-lora/imdb

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
!python ../../../peft_train.py \
--model_name google/flan-t5-xl \
--max_seq_len 2048 \
--group_by_length \
--max_steps 200 \
--dataset_name ../../../text-classification/imdb \
--num_labels 2 \
--epochs 5 \
--learning_rate 1e-3 \
--per_device_train_batch_size 64 \
--per_device_eval_batch_size 64 \
--model_type SEQ_2_SEQ_LM \
--output_model_path ./result/flan-t5-xl-imdb-lora \
--bnb_4bit_compute_dtype float16 \
--load_in_8bit \
--use_4b

# Load your adapter from the Hub

In [7]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

peft_model_id = "./result/flan-t5-cup-lora"
base_model_name_or_path = '.../../../pretrain_models/google-flan-t5-small'
config = PeftConfig.from_pretrained(peft_model_id)

# model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, torch_dtype="auto", device_map="auto")
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, torch_dtype="auto", device_map="cpu")
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

In [9]:
model.eval()
input_text = "In January-September 2009 , the Group 's net interest income increased to EUR 112.4 mn from EUR 74.3 mn in January-September 2008 ."
inputs = tokenizer(input_text, return_tensors="pt")

outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)

print("input sentence: ", input_text)
print(" output prediction: ", tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

input sentence:  In January-September 2009 , the Group 's net interest income increased to EUR 112.4 mn from EUR 74.3 mn in January-September 2008 .
 output prediction:  ['positive positive positive positive positive positive positive positive positive positive']


In [8]:
model.eval()
input_text = "In January-September 2009 , the Group 's net interest income increased to EUR 112.4 mn from EUR 74.3 mn in January-September 2008 ."
inputs = tokenizer(input_text, return_tensors="pt")

outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)

print("input sentence: ", input_text)
print(" output prediction: ", tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

input sentence:  In January-September 2009 , the Group 's net interest income increased to EUR 112.4 mn from EUR 74.3 mn in January-September 2008 .
 output prediction:  ['positive positive positive positive positive positive positive positive positive positive']


## 1.3 Experimental Result

In [ ]:
class HTMLRender:
    def __init__(self,html_str):
        self.html_str =html_str
    def _repr_html_(self):
       return self.html_str


In [ ]:
model_accurcy_html = '''
<table>
  <tr>
    <th>eval loss</th>
    <th>eval accuracy</th>
    <th>eval precision</th>
    <th>eval recall</th>
    <th>eval f1</th>
  </tr>
  <tr>
    <td style="background-color:#4C72B0;">0.102</td>
    <td style="background-color:#55A868;">0.896</td>
    <td style="background-color:#C44E52;">0.90</td>
     <td style="background-color:#8172B2;">0.896</td>
    <td style="background-color:#64B5CD;">0.894</td>
  </tr>

</table>

       '''
HTMLRender(model_accurcy_html)

In [ ]:
accuracy_html = '''
<img src="./image/flan-t5-small-accuracy.png" alt="flan-t5-small-accuracy" width="70%">
'''
HTMLRender(accuracy_html)

In [ ]:
## Load your adapter from the Hub
